<a href="https://colab.research.google.com/github/olonok69/LLM_Notebooks/blob/main/langchain/Langchain_Document_classification_Chromadb_Sentence_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.1 --progress-bar off
!pip install -qqq transformers==4.34.0 --progress-bar off
!pip install -qqq accelerate==0.23.0 --progress-bar off
!pip install -qqq bitsandbytes==0.41.1 --progress-bar off

In [ ]:
!pip install sentence-transformers spacy langchain trl datasets pypdf jq html2text  chromadb unstructured -q

# Chromadb

Chroma is the open-source embedding database. Chroma makes it easy to build LLM apps by making knowledge, facts, and skills pluggable for LLMs.

https://docs.trychroma.com/

https://python.langchain.com/docs/integrations/vectorstores/chroma


In [ ]:
# Import libraries
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer, AutoModelForCausalLM
import torch
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import transformers
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import chromadb
import pandas as pd
from chromadb.utils import embedding_functions



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data_path = "/content/drive/MyDrive/data/doc_classification"

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import DirectoryLoader

#embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L12-v2")

In [ ]:
model_name='sentence-transformers/gtr-t5-large'#'google/flan-t5-large'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Langchain DirectoryLoader

https://python.langchain.com/docs/modules/data_connection/document_loaders/file_directory

In [ ]:
txt_loader = DirectoryLoader(path = data_path , glob = '*.txt', recursive=True)
docs = txt_loader.load()

In [ ]:
len(docs)

In [ ]:
docs[0]

In [ ]:
# Text Summarization Pipeline
text_summarization_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="summarization",
    temperature=0.2,
    repetition_penalty=1.1,
      max_new_tokens=2000,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,


)
llm = HuggingFacePipeline(pipeline=text_summarization_pipeline)

In [ ]:
device

In [ ]:
for doc in docs:
    num_tokens = llm.get_num_tokens(doc.page_content)
    print (f"file{doc.metadata['source']} has {num_tokens} tokens")

In [ ]:
import pandas as pd
from chromadb.utils import embedding_functions
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

In [ ]:
purview_classes_path = "/content/drive/MyDrive/data/Microsoft_Purview_Classifiers_with_ddg_to_load.csv"

# Sentence Transformers

https://huggingface.co/sentence-transformers

# sentence-transformers/gtr-t5-large

https://huggingface.co/sentence-transformers/gtr-t5-large

In [ ]:
#chroma_client.delete_collection(name="purview_data")

In [ ]:


chroma_path = "/content/drive/MyDrive/chroma_persistence"
#delete collection if this exists



model_name = "sentence-transformers/gtr-t5-large"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)


#creating a new collection , cosine similarity it will be the metrics to measure similitude
#collection = chroma_client.create_collection(name="purview_data")#, metadata={"hnsw:space": "cosine"})

In [ ]:
from langchain.document_loaders import CSVLoader

# Load data from a CSV file using CSVLoader
loader = CSVLoader(purview_classes_path, metadata_columns=["classifier"])
documents = loader.load()


In [ ]:
documents[0].page_content


In [ ]:
vectordb = Chroma.from_documents(documents=documents,

                                 # Chose the embedding you want to use
                                 embedding=hf,
                                 collection_metadata = {"hnsw:space": "cosine"},
                                 persist_directory=chroma_path)

In [ ]:
vectordb.persist()
vectordb = None

In [ ]:
vectordb = Chroma(persist_directory=chroma_path,
                  embedding_function=hf

                  )

The similarity_search_with_score function in LangChain with Chroma DB returns higher scores for less relevant documents because it uses cosine distance as the scoring metric. In cosine distance, a lower score indicates a higher similarity between the query and the document. Therefore, documents with lower scores are more relevant to the query.

In [ ]:
res = vectordb.similarity_search(docs[100].page_content, k=5)

In [ ]:
for doc in res:
    print("-" * 80)
    print(doc.page_content)
    print("-" * 80)
    print(f"Class {doc.metadata.get('classifier')}")
    print("-" * 80)

In [ ]:
docs_with_score = vectordb.similarity_search_with_score(docs[100].page_content, k=5)

In [ ]:
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)
    print(f"Class {doc.metadata.get('classifier')}")
    print("-" * 80)

In [ ]:
docs[100].page_content

In [ ]:
vectordb.embeddings